In [1]:
import cv2
import numpy as np
import os

# Define your input and output directories
input_dir = 'data_p\img'
output_dir = 'dataug'
label_dir = 'data_p\lab'

# Make sure the output directories exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Function to perform data augmentation
def augment_data(image, bbox, keypoints):
    augmented_data = []

    # Original data
    #augmented_data.append((image, bbox, keypoints))

    # 1- Contrast +50
    contrast_image = cv2.convertScaleAbs(image, alpha=1.5, beta=0)
    contrast_bbox = bbox
    contrast_keypoints = [kp for kp in keypoints]
    augmented_data.append((contrast_image, contrast_bbox, contrast_keypoints))

    # 2- Contrast -50
    contrast_image = cv2.convertScaleAbs(image, alpha=0.5, beta=0)
    contrast_bbox = bbox
    contrast_keypoints = [kp for kp in keypoints]
    augmented_data.append((contrast_image, contrast_bbox, contrast_keypoints))

    # 3- Contrast +75
    contrast_image = cv2.convertScaleAbs(image, alpha=1.50, beta=-200)
    contrast_bbox = bbox
    contrast_keypoints = [kp for kp in keypoints]
    augmented_data.append((contrast_image, contrast_bbox, contrast_keypoints))

    # 4- Contrast -75
    contrast_image = cv2.convertScaleAbs(image, alpha=0.75, beta=-190)
    contrast_bbox = bbox
    contrast_keypoints = [kp for kp in keypoints]
    augmented_data.append((contrast_image, contrast_bbox, contrast_keypoints))
    
    # 5- Color Rojo
    red_image = image.copy()
    red_image[:, :, 1] = 0  # Set green channel to 0
    red_image[:, :, 2] = 0  # Set blue channel to 0
    red_bbox = bbox
    red_keypoints = [kp for kp in keypoints]
    augmented_data.append((red_image, red_bbox, red_keypoints))

    # 6- Color Verde
    green_image = image.copy()
    green_image[:, :, 0] = 0  # Set blue channel to 0
    green_image[:, :, 2] = 0  # Set red channel to 0
    green_bbox = bbox
    green_keypoints = [kp for kp in keypoints]
    augmented_data.append((green_image, green_bbox, green_keypoints))

    # 7- Color Purpura
    purple_image = image.copy()
    purple_image[:, :, 0] = 128  # Set blue channel to half intensity
    purple_image[:, :, 1] = 0    # Set green channel to 0
    purple_bbox = bbox
    purple_keypoints = [kp for kp in keypoints]
    augmented_data.append((purple_image, purple_bbox, purple_keypoints))

    # 8- Color Amarillo
    purple_image = image.copy()
    purple_image[:, :, 2] = 50 # Set blue channel to half intensity
    purple_image[:, 2, 1] = 128    # Set green channel to 0
    purple_bbox = bbox
    purple_keypoints = [kp for kp in keypoints]
    augmented_data.append((purple_image, purple_bbox, purple_keypoints))

    # 9- Color Azul
    purple_image = image.copy()
    purple_image[2, :, 0] = 128 # Set blue channel to half intensity
    purple_image[:, :, 0] = 0    # Set green channel to 0
    purple_bbox = bbox
    purple_keypoints = [kp for kp in keypoints]
    augmented_data.append((purple_image, purple_bbox, purple_keypoints))
    
    # 10- Contrast y Brillo +50
    contrast_image = cv2.convertScaleAbs(image, alpha=2.0, beta=-150)
    contrast_bbox = bbox
    contrast_keypoints = [kp for kp in keypoints]
    augmented_data.append((contrast_image, contrast_bbox, contrast_keypoints))
    
    # 11- Des-Texturizado
    kernel = np.array([[0, 1, 0],
                       [1, -3, 1],
                       [0, 1, 0]], dtype=np.float32)

    edge_enhanced_image = cv2.filter2D(image, cv2.CV_8U, kernel)
    edge_enhanced_bbox = bbox
    edge_enhanced_keypoints = [kp for kp in keypoints]
    augmented_data.append((edge_enhanced_image, edge_enhanced_bbox, edge_enhanced_keypoints))
    
    # 12- Convertir la imagen a escala de grises
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    sobel_x = cv2.Sobel(gray_image, cv2.CV_64F, 1, 0, ksize=3)
    sobel_y = cv2.Sobel(gray_image, cv2.CV_64F, 0, 1, ksize=3)
    magnitude = cv2.magnitude(sobel_x, sobel_y)
    salient_edge_map = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)

    salient_edge_bbox = bbox
    salient_edge_keypoints = [kp for kp in keypoints]
    augmented_data.append((salient_edge_map, salient_edge_bbox, salient_edge_keypoints))
    
    # 13- Contrast y Brillo +50
    contrast_image = cv2.convertScaleAbs(image, alpha=1.5, beta=50)
    contrast_bbox = bbox
    contrast_keypoints = [kp for kp in keypoints]
    augmented_data.append((contrast_image, contrast_bbox, contrast_keypoints))

    # 14- Contrast y Brillo -50
    contrast_image = cv2.convertScaleAbs(image, alpha=0.5, beta=-50)
    contrast_bbox = bbox
    contrast_keypoints = [kp for kp in keypoints]
    augmented_data.append((contrast_image, contrast_bbox, contrast_keypoints))

    # 15- Decolorar la imagen
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    decolor_bbox = bbox
    decolor_keypoints = [kp for kp in keypoints]
    augmented_data.append((gray_image, decolor_bbox, decolor_keypoints))

    return augmented_data

# Process each image and its corresponding annotation
for filename in os.listdir(input_dir):
    if filename.endswith(".jpg"):
        image_path = os.path.join(input_dir, filename)
        label_path = os.path.join(label_dir, filename[:-4] + '.txt')

        # Read image
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Read label file
        with open(label_path, 'r') as label_file:
            labels = label_file.readline().split()

        # Extract bounding box and keypoints from labels
        bbox = list(map(float, labels[1:5]))
        keypoints = list(map(float, labels[5:]))

        # Perform data augmentation
        augmented_data_list = augment_data(image, bbox, keypoints)

        # Save augmented images and labels
        for i, (augmented_image, augmented_bbox, augmented_keypoints) in enumerate(augmented_data_list):
            augmented_image_path = os.path.join(output_dir, f'aug_{i}_{filename}')
            cv2.imwrite(augmented_image_path, cv2.cvtColor(augmented_image, cv2.COLOR_RGB2BGR))

            augmented_label_path = os.path.join(output_dir, f'aug_{i}_{filename[:-4]}.txt')
            with open(augmented_label_path, 'w') as augmented_label_file:
                augmented_label_file.write('0 {} {} {} {} '.format(*augmented_bbox))
                augmented_label_file.write(' '.join(map(str, augmented_keypoints)))

print("Data augmentation complete.")



Data augmentation complete.


In [ ]:
import cv2
import numpy as np
import os

def draw_bbox_and_keypoints(image,  keypoints, color=(255, 0, 0)):

    # Draw keypoints
    for i in range(0, len(keypoints), 2):
        kp_x = int(keypoints[i] * image.shape[1])
        kp_y = int(keypoints[i + 1] * image.shape[0])
        cv2.circle(image, (kp_x, kp_y), 3, color, -1)

    return image

# Define your input directory
input_dir = 'golpes_3_340\images'

label_dir = 'golpes_3_340\labels'

# Process each image and its corresponding annotation
for filename in os.listdir(input_dir):
    if filename.endswith(".jpg"):
        image_path = os.path.join(input_dir, filename)
        label_path = os.path.join(label_dir, filename[:-4] + '.txt')

        # Read image
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Read label file
        with open(label_path, 'r') as label_file:
            labels = label_file.readline().split()

        # Extract bounding box and keypoints from labels
        bbox = list(map(float, labels[1:5]))
        keypoints = list(map(float, labels[5:]))

        # Draw bounding box and keypoints on the image
        image_with_annotations = draw_bbox_and_keypoints(image.copy(), keypoints)

        # Display the result
        cv2.imshow('Annotations Verification', cv2.cvtColor(image_with_annotations, cv2.COLOR_RGB2BGR))
        cv2.waitKey(0)
        cv2.destroyAllWindows()
